# ADY201m - Fall 2024 - Lab 1

### Your roll number: SE190168
### Your full name: Nguyễn Trương Phúc Thịnh

## Datascience and SQL

In [3]:
# To get your stock symbol, you should query from db AI1801.db to know your stock symbol and trading data.

# !HINT: use pragma table_info(table_name) to get table structure
# !HINT: stock symbols are assigned to students by ID

# Provide your code/query to get your stock symbol and company information.
with connect('AI1801.db') as connection:
    cursor = connection.cursor()
    symbol = cursor.execute(
        """
        SELECT StockList.Symbol
        FROM StudentList
            JOIN SymbolAssignment ON StudentList.StudentID = SymbolAssignment.StudentID
            JOIN StockList ON SymbolAssignment.SymbolID = StockList.SymbolID
        WHERE StudentList.RollNumber = 'SE190168'
        """
    ).fetchone()[0]
    print(symbol)

# Fill your stock symbol bellow
symbol = "PVS"


PVS


## Business understanding
- Company name: Tổng công ty Cổ phần Dịch vụ Kỹ thuật Dầu khí Việt Nam
- Category name: Dịch vụ dầu khí, công nghiệp.

 Tổng công ty Cổ phần Dịch vụ Kỹ thuật Dầu khí Việt Nam (PTSC) là tổng công ty hàng đầu trong lĩnh vực cung cấp các dịch vụ kỹ thuật dầu khí, công nghiệp tại Việt Nam, và là thương hiệu lớn trên thị trường dịch vụ dầu khí, công nghiệp trong khu vực.

Lĩnh vực hoạt động chính của PTSC là cung cấp các loại hình dịch vụ kỹ thuật cho các ngành dầu khí, năng lượng, công nghiệp, trong đó có nhiều loại hình dịch vụ mang tính chiến lược, chất lượng cao, mũi nhọn, đã phát triển và được chuyên nghiệp hóa đạt trình độ quốc tế như: EPCI công trình biển; EPC công trình công nghiệp; Kho nổi chứa, xử lý và xuất dầu thô FSO/FPS0; Tàu dịch vụ dầu khí; Khảo sát địa chấn, địa chất và công trình ngầm; Lắp đặt, vận hành và bảo dưỡng công trình biển; Cảng dịch vụ, Dịch vụ cung ứng nhân lực kỹ thuật và các Dịch vụ cho dự án năng lượng tái tạo.

Nguồn thông tin: [Trang giới thiệu chính thức của PTSC](https://www.ptsc.com.vn/gioi-thieu)

## Working with data
Calculate the max price, min price, open price, close price, total trading volume on ...

In [5]:
# Calculate the max price, min price, open price, close price , total trading volume of you stock symbol on August 20th. You should get data from the AI1801.db.

query = cursor.execute(
    """
    WITH Stock AS (
        SELECT
            cast(price AS real) AS price,
            cast(volume AS int) AS volume,
            row_number() OVER (ORDER BY tradeDate) AS rowNumAsc,
            row_number() OVER (ORDER BY tradeDate DESC) AS rowNumDesc
        FROM StockTrading
        WHERE symbol = 'PVS' AND substring(tradeDate, 0, 11) = '2024-08-20'
    )
    SELECT
        max(price),
        min(price),
        (SELECT price FROM Stock WHERE rowNumAsc = 1),
        (SELECT price FROM Stock WHERE rowNumDesc = 1),
        sum(volume)
    FROM Stock
    """
).fetchone()

print(f"Max price: {query[0]}",
      f"Min price: {query[1]}",
      f"Open price: {query[2]}",
      f"Close price: {query[3]}",
      f"Total volume: {query[4]}",
      sep='\n')


Max price: 40.3
Min price: 36.3
Open price: 36.3
Close price: 40.3
Total volume: 17412


## Data collection and visualization
* Visualize your stock trading data from September 15th to September 30th 


In [7]:

# !HINT: You can get data from this link, print to show your link!
url = "https://s.cafef.vn/Ajax/PageNew/DataHistory/PriceHistory.ashx?Symbol={0}&StartDate=09/15/2024&EndDate=09/30/2024".format('PVS')
print(url)

from requests import get
from pandas import DataFrame, Timestamp, to_datetime

parsed_data = get(url).json()['Data']
data = DataFrame(parsed_data['Data'])
data['Ngay'] = to_datetime(data['Ngay'], format='%d/%m/%Y')

https://s.cafef.vn/Ajax/PageNew/DataHistory/PriceHistory.ashx?Symbol=PVS&StartDate=09/15/2024&EndDate=09/30/2024


### Your line plot of close prices from September 15th to September 30th

In [12]:
from plotly.express import line, bar

start_date = Timestamp('2024-09-15')
end_date = Timestamp('2024-09-30')

closing_prices = data[(data['Ngay'] >= start_date) & (data['Ngay'] <= end_date)]

figure = line(closing_prices, x='Ngay', y='GiaDongCua', title='Close Prices from September 15th to 30th')
figure.show()

- What story can you tell about this plot?

1. Xu hướng chung:
Giá đóng cửa có xu hướng tăng nhẹ từ giữa tháng 9 và sau đó giảm dần về cuối tháng.
Giá khởi đầu ở mức 40.0 vào ngày 16/9 và kết thúc ở mức 40.6 vào ngày 30/9.

2. Điểm cao nhất và thấp nhất:
Giá cao nhất được ghi nhận là 41.6 vào các ngày 20/9, 24/9 và 25/9.
Giá thấp nhất là 40.0 vào các ngày 16/9 và 17/9.


3. Biến động đáng chú ý:
Có một sự tăng giá đáng kể từ 40.4 lên 41.6 (tăng 2.97%) vào ngày 20/9.
Sau đó, giá duy trì ở mức cao trong vài ngày trước khi bắt đầu giảm dần.


4. Giai đoạn ổn định: Từ ngày 23/9 đến 25/9, giá duy trì ổn định ở mức 41.3 - 41.6.

5. Xu hướng cuối kỳ: Từ ngày 26/9 đến 30/9, giá có xu hướng giảm dần, từ 41.2 xuống 40.6.


6. Tổng kết:
Nhìn chung, mặc dù có sự tăng giá đáng kể vào giữa tháng, giá đóng cửa cuối cùng (40.6) chỉ cao hơn một chút so với giá đầu kỳ (40.0).
Điều này cho thấy thị trường có một số biến động trong tháng nhưng về cơ bản vẫn duy trì được sự ổn định tương đối.

### Your bar of trading volumes from September 15th to September 30th 

In [8]:
start_date = Timestamp('2024-09-15')
end_date = Timestamp('2024-09-30')

trading_volumes = data[(data['Ngay'] >= start_date) & (data['Ngay'] <= end_date)]

figure = bar(trading_volumes, x='Ngay', y='KhoiLuongKhopLenh', title='Trading Volumes from September 15th to 30th')
figure.show()

1. Xu hướng chung:
Khối lượng giao dịch biến động mạnh trong khoảng thời gian này.
Có một số ngày với khối lượng giao dịch rất cao, đặc biệt là vào giữa và cuối tháng.


2. Điểm cao nhất và thấp nhất:
Khối lượng giao dịch cao nhất được ghi nhận vào ngày 20/9 với 7,407,148 đơn vị.
Khối lượng thấp nhất là vào ngày 18/9 với 976,866 đơn vị.


3. Biến động đáng chú ý:
Có sự tăng đột biến vào ngày 20/9, với khối lượng giao dịch tăng gần gấp 7 lần so với ngày trước đó.
Sau ngày 20/9, khối lượng giao dịch giảm xuống nhưng vẫn duy trì ở mức cao hơn so với đầu tháng.


4. Giai đoạn biến động mạnh:
Từ ngày 24/9 đến 26/9, khối lượng giao dịch tăng liên tục và đạt mức cao thứ hai trong khoảng thời gian này vào ngày 26/9 với 5,812,652 đơn vị.


5. Xu hướng cuối kỳ:
Khối lượng giao dịch có xu hướng giảm dần trong ba ngày cuối của tháng 9, nhưng vẫn duy trì ở mức tương đối cao so với đầu tháng.


6. Tổng kết:
Khối lượng giao dịch trung bình trong khoảng thời gian này là khoảng 2,972,687 đơn vị mỗi ngày.
Có sự biến động mạnh trong khối lượng giao dịch, với một số ngày có khối lượng giao dịch rất cao, cho thấy có thể có những sự kiện hoặc tin tức quan trọng ảnh hưởng đến thị trường vào những ngày này.